## analyse statistique des données :



In [ ]:
import os
import tempfile
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import time
#import scikitplot as skplt
import warnings
warnings.filterwarnings("ignore")
sns.set()
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
circuits = pd.read_csv("datasets/circuits.csv", index_col=0)
constructorResults = pd.read_csv("datasets/constructor_results.csv", index_col=0)
constructors = pd.read_csv("datasets/constructors.csv", index_col=0)
constructorStandings = pd.read_csv("datasets/constructor_standings.csv", index_col=0)
drivers = pd.read_csv("datasets/drivers.csv", index_col=0)
lapTimes = pd.read_csv("datasets/lap_times.csv")
pitStops = pd.read_csv("datasets/pit_stops.csv")
qualifying = pd.read_csv("datasets/qualifying.csv", index_col=0)
races = pd.read_csv("datasets/races.csv",index_col=0)
results = pd.read_csv("datasets/results.csv", index_col=0)
status = pd.read_csv("datasets/status.csv", index_col=0)
tyres = pd.read_csv("datasets/tyres.csv")
races = races.reindex(np.random.permutation(races.index))

In [ ]:
def getDriverName(driverId):
    '''
    In : driverId (int) : numéro d'identification du pilote
    Out : nom et prénom du pilote (string)
    '''
    try:
        name = str(drivers['forename'].loc[driverId])+' '+str(drivers['surname'].loc[driverId])
        return name
    except UnicodeDecodeError:
        return driverRef

def getDriverId(forename, surname):
    '''
    In : prénom (string) et nom (string)
    Out : driverId (int)
    '''
    return drivers.index[(drivers['forename'] == forename) & (drivers['surname'] == surname)].tolist()[0]

def getConstructorId(constructor):
    '''
    In : constructor (string) : exemple, "McLaren" ou "Ferrari"
    Out : constructorId (int)
    '''
    return constructors.index[constructors['name'] == constructor].tolist()[0]

def getConstructorName(id):
    '''
    In : constructorId (int)
    Out : nom du constructeur (string)
    '''
    return str(constructors['name'].loc[id])

def getDriversInRace(raceId):
    '''
    In : raceId (int)
    Out : les id des pilotes ayant participé à la course (list(int))
    '''
    drivers = results[results['raceId'] == raceId]
    drivers = drivers['driverId']
    return drivers.tolist()

def getTeammate(raceId,driverId):
    result = results[results['raceId'] == raceId]
    constructor = result[result['driverId'] == driverId]['constructorId'].iloc[0]
    current_round = races['round'].loc[raceId]
    try:
        teammate = result[(results['constructorId'] == constructor) & (results['driverId'] != driverId)]['driverId'].iloc[0]
    except Exception:
        if current_round > 1: #Si ce n'est pas la première course de la saison, on retourne l'équipier de la course précédente
            return getTeammate(raceId-1, driverId)
        else:
            return getTeammate(raceId+1, driverId)
    return teammate

In [ ]:

def getInformation(raceId, driverId):
    '''
    Cette fonction permet d'exécuter quelques lignes de code fastidieuses.
    In : raceId (int)
         driverId (int)
    Out : 6 DataFrames correspondant à l'union de chaque DataFrame importé avec le numéro de course et le numéro de pilote.
    '''
    try:
        race = races.loc[raceId] #Find the race
        driver = drivers.loc[driverId] #Find the driver
        result = results[(results['raceId'] == raceId) & (results['driverId'] == driverId)].iloc[0]
        pits = pitStops[(pitStops['raceId'] == raceId) & (pitStops['driverId'] == driverId)]
        quali = qualifying[(qualifying['raceId'] == raceId) & (qualifying['driverId'] == driverId)]
        laps = lapTimes[(lapTimes['raceId'] == raceId) & (lapTimes['driverId'] == driverId)]
        return(race,driver,result,pits,quali,laps)
    except Exception: #Si on a entré un mauvais couple (raceId, driverId)
        print('Impossible de trouver la course n°{}'.format(raceId))
        return getInformation(raceId-1,driverId)

def getRaceInformation(raceId):
    '''
    Cette fonction d'affichage imprime un résumé de la course, avec l'identifiant, le vanqueur et le podium.
    In : raceId (int)
    Out : rien 
    '''
    try:
        race = races.loc[raceId]
        result = results[results['raceId'] == raceId]
        print(result)
        winner = result[result['positionOrder'] == 1]['driverId'].iloc[0]
        print(str(race['year'])+ ' '+str(race['name'])+'\n')
        print('Le vainqueur de la course est : '+getDriverName(winner))
        print('Le podium est composé de :')
        for i in range(0,3):
            print(str(getDriverName(result['driverId'].iloc[i]))+' ('+str(result['driverId'].iloc[i])+') ')
        print('')
    except Exception:
        print('Impossible de trouver la course n°{}'.format(raceId))
        return getRaceInformation(raceId-1)

def getQualifyingPosition(raceId, driverId):
    '''
    In : raceId (int)
         driverId (int)
    Out : le résultat de qualifications du pilote (pas forcément égal à la position sur la grille, il peut 
          y avoir des pénalités) (int)    
    '''
    quali = getInformation(raceId,driverId)[4]
    if not quali.empty:
        return quali['position'].iloc[0]
    else :
        return 'Inconnu'
    
def getFastestLapTime(raceId):
    '''
    In : raceId (int)
    Out : le temps en millisecondes du tour de course le plus rapide (int)
    '''
    result = results[results['raceId'] == raceId]
    mylap = result['fastestLapTime'].min()
    minutes = int(mylap.split(":")[0])
    seconds = int(mylap.split(":")[1].split(".")[0])
    milliseconds = int(mylap.split(":")[1].split(".")[1])
    return (60000*minutes+1000*seconds+milliseconds)

def getQualifyingTime(raceId,driverId):
    '''
    In : raceId (int)
         driverId (int)
    Out : le temps en millisecondes du tour de qualifications le plus rapide (int)
    '''
    try:
        quali = qualifying[(qualifying['raceId'] == raceId) & (qualifying['driverId'] == driverId)]
        quali = quali.fillna(value='9:99.999')
        quali = quali.iloc[0]
        mylap = quali[['q1','q2','q3']]
        mylap = mylap.min()
        minutes = int(mylap.split(":")[0])
        seconds = int(mylap.split(":")[1].split(".")[0])
        milliseconds = int(mylap.split(":")[1].split(".")[1])
        return (60000*minutes+1000*seconds+milliseconds)
    except Exception: #Le pilote n'a pas participé aux qualifications (exemple Albon à Shanghai 2019)
        return getQualifyingTime(raceId, getTeammate(raceId,driverId)) #On retourne le temps de qualifications de son coéquipier.

def getRaceReport(raceId, driverId):
    '''
    Cette fonction d'affichage imprime un résumé de course pour un pilote passé en paramètre.
    On imprime le résultat de qualifications, la position sur la grille, la position finale,
    les pit-stops effectués par le pilote si l'information est disponible, on trace un graphique
    de la position tour par tour et du temps au tour.
    In : raceId (int)
         driverId (int)
    '''
    (race,driver,result,pits,quali,laps) = getInformation(raceId,driverId)
    print('{} {} (raceId : {})\n'.format(race['year'],race['name'],race.index.name))
    print('Résumé de course de '+str(driver['forename'])+' '+str(driver['surname'])+'\n')
    print('Résultat de qualification : '+str(getQualifyingPosition(raceId,driverId)))
    print('Position sur la grille de départ : '+str(result['grid'])+'\n')
    print('Position finale : '+str(result['positionText']))
    print('Pit-stops effectués par le pilote : '+str(pits.shape[0]))
    if not pits.empty:   
        display(pits[['lap','duration']])
    if not laps.empty:
        fig = plt.figure(figsize=(8,6))
        ax1 = fig.add_subplot(111)
        plt.fill_between(laps['lap'], laps['position'],color="gold", alpha=0.5)
        ax1.plot(laps['lap'].tolist(), laps['position'].tolist(),color='yellow')
        ax1.set_ylabel('position')
        ax1.set_facecolor('xkcd:white')
        ax2 = ax1.twinx()
        ax2.plot(laps['lap'].tolist(), laps['milliseconds'].tolist(), 'blue')
        ax2.set_ylabel('milliseconds',color='blue')
        ax1.grid(False)
        ax2.grid(False)

def getComparison(raceId,driver1,driver2):
    '''
    Cette fonction d'affichage imprime un head-to-head entre deux pilotes passés en paramètres.
    On trace la comparaison de la position ainsi que la comparaison des temps au tour.
    In : raceId (int)
         driver1 (int)
         driver2 (int)
    '''
    (race,driver,result,pits,quali,laps) =getInformation(raceId,driver1)
    (race2,driver2,result2,pits2,quali2,laps2) =getInformation(raceId,driver2)
    if not laps.empty and not laps2.empty:
        fig = plt.figure(figsize=(14,5),dpi=300)
        ax2 = fig.add_subplot(121)
        l1, = ax2.plot(laps['lap'].tolist(), laps['milliseconds'].tolist(), 'blue')
        l2, = ax2.plot(laps2['lap'].tolist(), laps2['milliseconds'].tolist(), 'red')
        ax2.set_ylabel('milliseconds')
        l1.set_label(getDriverName(driver1))
        l2.set_label(getDriverName(driver2.name))
        ax2.legend()
        ax2.title.set_text('Comparaison des temps au tour')
        
        ax1 = fig.add_subplot(122)
        ax1.set_ylabel('position')
        plt.gca().invert_yaxis()
#         plt.fill_between(laps['lap'], laps['position'],color="blue", alpha=0.1)
#         plt.fill_between(laps2['lap'], laps2['position'],color="red", alpha=0.1)
        l3, = ax1.plot(laps['lap'].tolist(), laps['position'].tolist(),color='blue')
        l3.set_label(getDriverName(driver1))
        l4, = ax1.plot(laps2['lap'].tolist(), laps2['position'].tolist(),color='red')
        l4.set_label(getDriverName(driver2.name))
        ax1.legend()
        ax1.title.set_text('Comparaison de la position')
        plt.tight_layout()
    else:
        print('Comparaison non disponible')